In [7]:
# Лабораторная работа №6 - Импорты
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [17]:
def classification_training(data):
    """
    Функция для обучения модели Random Forest с подбором гиперпараметров

    Args:
        data: DataFrame с данными
    """
    print("Запуск функции classification_training для ЛР6...")
    print(f"Размер полученных данных: {data.shape}")

    # 1. Трансформация целевой переменной в бинарную
    print("\n1. Трансформация целевой переменной...")

    # Создаем копию данных чтобы не изменять оригинал
    df = data.copy()

    # Используем медиану для бинарной классификации
    median_value = df['mental_wellness_index_0_100'].median()
    print(f"Используем медиану для бинарной классификации: {median_value:.4f}")

    # Трансформируем mental_wellness_index_0_100 в бинарную переменную
    df['target'] = df['mental_wellness_index_0_100'].apply(lambda x: 0 if x < median_value else 1)

    # Удаляем исходный столбец mental_wellness_index_0_100
    df = df.drop('mental_wellness_index_0_100', axis=1)

    print(f"Распределение классов:")
    print(f"   Класс 0 (<медианы): {sum(df['target'] == 0)} samples")
    print(f"   Класс 1 (>=медианы): {sum(df['target'] == 1)} samples")

    # 2. Подготовка данных для обучения
    print("\n2. Подготовка данных для обучения...")

    # Разделяем на признаки (X) и целевую переменную (y)
    X = df.drop('target', axis=1)
    y = df['target']

    print(f"   Признаки (X): {X.shape}")
    print(f"   Целевая переменная (y): {y.shape}")

    # Разделяем на тренировочную и тестовую выборки (70%/30%)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=42, stratify=y
    )

    print(f"   Тренировочная выборка: {X_train.shape[0]} samples")
    print(f"   Тестовая выборка: {X_test.shape[0]} samples")

    # 3. Создание словаря-сетки для GridSearchCV
    print("\n3. Создание сетки гиперпараметров для GridSearchCV...")

    # Разумные значения гиперпараметров (не больше количества samples)
    param_grid = {
        'n_estimators': [50, 100, 150],  # количество деревьев (меньше чем samples)
        'max_depth': [3, 5, 7],          # глубина деревьев (предотвращает переобучение)
        'min_samples_split': [2, 5, 10], # минимальное samples для разделения
        'min_samples_leaf': [1, 2, 4]    # минимальное samples в листе
    }

    print("   Сетка гиперпараметров:")
    for param, values in param_grid.items():
        print(f"   {param}: {values}")

    # 4. Создание экземпляра Random Forest
    print("\n4. Создание модели Random Forest...")
    rf_model = RandomForestClassifier(random_state=42)

    # 5. Поиск оптимальных гиперпараметров с GridSearchCV
    print("\n5. Поиск оптимальных гиперпараметров...")
    grid_search = GridSearchCV(
        estimator=rf_model,
        param_grid=param_grid,
        cv=3,  # 3-кратная кросс-валидация
        scoring='f1',  # оптимизируем по f1-score
        n_jobs=-1  # используем все ядра процессора
    )

    # Обучение GridSearchCV
    grid_search.fit(X_train, y_train)

    # Вывод оптимальных параметров
    print("   Оптимальные гиперпараметры:")
    for param, value in grid_search.best_params_.items():
        print(f"   {param}: {value}")

    # 6. Обучение финальной модели с оптимальными параметрами
    print("\n6. Обучение финальной модели...")
    best_rf_model = grid_search.best_estimator_

    # Предсказания на тестовой выборке
    y_pred = best_rf_model.predict(X_test)

    # 7. Вычисление метрик
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # 8. Вывод результатов по заданному шаблону
    print("\n7. Результаты классификации:")
    print(f"RF: {accuracy:.4f}; {f1:.4f}")

    print("\nФункция завершена успешно!")

In [18]:
# Загрузка и тестирование на реальных данных
print("=== ТЕСТИРОВАНИЕ НА РЕАЛЬНЫХ ДАННЫХ ===")

from google.colab import files

print("Загрузите файл 'processed_mental_health_data.csv'")
uploaded = files.upload()

# Проверяем загрузку
file_names = list(uploaded.keys())
print(f"Загруженные файлы: {file_names}")

# Загружаем данные
file_name = file_names[0]
real_data = pd.read_csv(file_name)

print(f"✅ Файл '{file_name}' успешно загружен!")
print(f"Размер данных: {real_data.shape}")

# Запускаем функцию на реальных данных
print("\n=== ЗАПУСК НА РЕАЛЬНЫХ ДАННЫХ ===")
classification_training(real_data)

=== ТЕСТИРОВАНИЕ НА РЕАЛЬНЫХ ДАННЫХ ===
Загрузите файл 'processed_mental_health_data.csv'


Saving processed_mental_health_data.csv to processed_mental_health_data (1).csv
Загруженные файлы: ['processed_mental_health_data (1).csv']
✅ Файл 'processed_mental_health_data (1).csv' успешно загружен!
Размер данных: (379, 20)

=== ЗАПУСК НА РЕАЛЬНЫХ ДАННЫХ ===
Запуск функции classification_training для ЛР6...
Размер полученных данных: (379, 20)

1. Трансформация целевой переменной...
Используем медиану для бинарной классификации: -0.2638
Распределение классов:
   Класс 0 (<медианы): 189 samples
   Класс 1 (>=медианы): 190 samples

2. Подготовка данных для обучения...
   Признаки (X): (379, 19)
   Целевая переменная (y): (379,)
   Тренировочная выборка: 265 samples
   Тестовая выборка: 114 samples

3. Создание сетки гиперпараметров для GridSearchCV...
   Сетка гиперпараметров:
   n_estimators: [50, 100, 150]
   max_depth: [3, 5, 7]
   min_samples_split: [2, 5, 10]
   min_samples_leaf: [1, 2, 4]

4. Создание модели Random Forest...

5. Поиск оптимальных гиперпараметров...
   Оптимальн